# MNIST Model From "Scratch"

Goal: To reproduce the second half of Chapter 4 from the Fast.ai book with as little referencing the material as possible. Looking up things in the documentation is allowed.

## Data Exploration

### Steps
- get data into tensors
- initialize params (weights + bias(es))
- multiply params by pixels (feed forward)
- measure loss
- get gradients
- step with gradients and learning rate

In [2]:
from fastai.data.external import *
import torch

In [3]:
data = untar_data(URLs.MNIST_SAMPLE)

In [29]:
from pathlib import Path

Path.BASE_PATH = data

train3s = data/'train'/'3'
print(train3s.ls()[0])

train7s = data/'train'/'7'
print(train7s.ls()[0])

/home/carver/.fastai/data/mnist_sample/train/3/47123.png
/home/carver/.fastai/data/mnist_sample/train/7/7420.png


In [30]:
from PIL import Image

image = Image.open(train3s.ls()[0])
image.show()
image7 = Image.open(train7s.ls()[0])
image7.show()

### Array_Interfaces and Fastbook's Tensor Function

This bit of code below (more or less copied from the [04_mnist_basics notebook](https://github.com/fastai/fastbook/blob/master/04_mnist_basics.ipynb)) took a while to comprehend. It turns out that the `array` method is a numpy construct and the `tensor` method is a fastbook construct.

The `array` method uses the [numpy __array_interface__ protocol](https://numpy.org/doc/stable/reference/arrays.interface.html) and [the `tensor` method](https://docs.fast.ai/torch_core.html#tensor) uses the same thing to turn this image into a 2D array, but then calls [torch.as_tensor](https://pytorch.org/docs/stable/generated/torch.as_tensor.html) to create a PyTorch Tensor object.

In [31]:
from numpy import *
print(array(image)[4:10, 4:10])

from fastbook import tensor
print(tensor(image)[4:10, 4:10])

[[  0   0   0 211 254 187]
 [  0   0   0 122 214 253]
 [  0   0   0   0   8  17]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   0   0]]
tensor([[  0,   0,   0, 211, 254, 187],
        [  0,   0,   0, 122, 214, 253],
        [  0,   0,   0,   0,   8,  17],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0]], dtype=torch.uint8)


In [32]:
sample_tens = tensor(image)
sample_tens.shape

torch.Size([28, 28])

In [73]:
# Get all the training images into individual tensors, then stack them.

all_valid_3s = [tensor(Image.open(path)) for path in train3s.ls()]
stacked_valid_3s = torch.stack(all_valid_3s).float()/255
print(stacked_valid_3s.shape)

all_valid_7s = [tensor(Image.open(path)) for path in train7s.ls()]
stacked_valid_7s = torch.stack(all_valid_7s).float()/255
print(stacked_valid_7s.shape)

torch.Size([6131, 28, 28])
torch.Size([6265, 28, 28])


### Now we want to create a list of tuples (x, y) such that x is an array of pixels and y is 0 or 1 (where 1 signifies that the image is a 3).

In [75]:

valid_3s_count = stacked_valid_3s.shape[0]
stacked_valid_3s = torch.reshape(stacked_valid_3s, (valid_3s_count,-1))
print(stacked_valid_3s.shape)

valid_7s_count = stacked_valid_7s.shape[0]
stacked_valid_7s = torch.reshape(stacked_valid_7s, (valid_7s_count,-1))
print(stacked_valid_7s.shape)

torch.Size([6131, 784])
torch.Size([6265, 784])


In [93]:
x = torch.cat([stacked_valid_3s, stacked_valid_7s])
x.shape, x[0, 210:220]

(torch.Size([12396, 784]),
 tensor([0.0000, 0.0000, 0.0196, 0.2235, 0.9922, 0.9843, 0.6235, 0.0000, 0.0000, 0.0000]))

In [94]:
# Couple ways to do this bit...
# y = torch.cat([tensor([1 for i in range(valid_3s_count)]), tensor([0 for i in range(valid_7s_count)])])
y = torch.cat([tensor([1] * valid_3s_count), tensor([0] * valid_7s_count)]).unsqueeze(1)
y.shape

torch.Size([12396, 1])

In [95]:
x_y = list(zip(x, y))

In [125]:
# Create tensor with initialized weights and bias, set up for getting gradients.
from numpy.random import normal

weights = tensor(normal(size=784)).requires_grad_()
print(weights.shape)
print(weights[0:4])
bias = tensor(normal(size=1)).requires_grad_()
print(bias)

torch.Size([784])
tensor([ 1.0893,  0.3820, -2.3425,  0.2997], grad_fn=<SliceBackward0>)
tensor([-1.7377], requires_grad=True)


Now I'm lost...

At this point, I have all the x and y inputs in tensors and the weights and bias in tensors (params).

I can't remember how things were done from here on out though...

I _think_ that there was an example done with just one image, doing the loss, gradient, and step. Then something was done with the matrix multiply operator in Python (`@`). I guess we just did the same steps there, on the CPU, before moving to the GPU...

### Okay, I'm lost. Looking at the Reference Now.

Here's what I missed:
- Forgot to normalize the image arrays.
- Need to list the zip (a list of a zip is a common python idiom).
- Forgot to unsqueeze the training y tensor (this probably would have been obvious if I did the list of zip and inspected).
- Used a reshape instead of a view (a reshape may or may not return a new tensor, whereas a view does not move the underlying data) -- not sure this is a mistake, per se.
- I used the param version of requires_grad, which is fine for the way I initialized the params. But init_params in the chapter multiplies by a std number before calling the requires_grad_() method.

### Now I've read through the reference again. Trying to continue it without referencing further...

In [126]:
def linear1(x, weights, bias): return x@weights + bias

In [127]:
# Let me see if I can do the torch.where thing...
preds = tensor([0.2, 0.5, 0.8])
targets = tensor([1, 1, 0])
torch.where(targets==1, 1-preds, preds)

# Had to look a bit (of course), but got it pretty close the first time. Main thing was that I forgot to make preds and targets tensors (just had them as arrays).

tensor([0.8000, 0.5000, 0.8000])

In [128]:
# Now let's try to get a mini-batch going...

from fastai.data.load import DataLoader

dl = DataLoader(dataset=x_y, batch_size=4, shuffle=True)
batch_x, batch_y = dl.one_batch()
batch_x, batch_y

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1],
         [0],
         [1],
         [1]]))

In [129]:
weights.shape

torch.Size([784])

In [130]:
predictions = linear1(batch_x, weights, bias)
predictions

tensor([ 7.9452, 11.2940, -5.3258,  9.3484], grad_fn=<AddBackward0>)

In [131]:
# Had to look up this function.
def sigmoid(x): return 1/(1+torch.exp(-x))

In [132]:
predictions = sigmoid(predictions)
predictions

tensor([0.9996, 1.0000, 0.0048, 0.9999], grad_fn=<MulBackward0>)

In [133]:
def mnist_loss(targets, predictions): return torch.where(targets==1, 1-predictions, predictions).mean()

In [134]:
loss = mnist_loss(batch_y, predictions)
loss

tensor(0.3745, grad_fn=<MeanBackward0>)

In [123]:
loss.backward()

In [124]:
weights.grad, bias.grad

/tmp/ipykernel_1139/18095485.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642969563/work/build/aten/src/ATen/core/TensorBody.h:480.)
  weights.grad, bias.grad


(None, None)

### I guess the requires_grad_() error I had made _was_ important...

Going back and trying again, leaving the above error...

In [135]:
loss.backward()

In [137]:
weights.grad.shape, weights.grad.mean(), bias.grad

(torch.Size([784]), tensor(-7.0804e-05), tensor([-0.0007]))

### Okay, I got a grad, but the shapes are different from the example.

The book's `weights.grad.shape` was `[784,1]`, but I'm just getting `[784]`...

I'm super tired at this point, so I'm just going to finish this with referencing.

In [139]:
def calc_grad(xb, yb, model):
    preds = model(xb, weights, bias)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [144]:
# Not sure if there's a difference between these two...
weights.grad = None
bias.grad = None

# weights.grad.zero_()
# bias.grad.zero_()

In [145]:
def train_epoch(model, lr, weights, bias):
    for xb, yb in dl:
        calc_grad(xb, yb, linear1)
        weights.data -= weights.grad * lr
        weights.grad.zero_()
        bias.data -= bias.grad *lr
        bias.grad.zero_()

In [146]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [148]:
batch_accuracy(linear1(batch_x, weights, bias), batch_y)

tensor(0.6250)

### Stopping Here. I'm totally exhausted. 

I'll get 'em next time.